**Instructions:** 
This notebook can be run by first executing the cells by pressing shift+enter while
selected or clicking the play button on the left. The GUI widgets then become
active and can be adjusted to reveal relationships betweeen box symmetry
and degeneracy for the 3-dimensional particle in a box (PIB) model system.
Start with the 'Install/Import Packages' cell and proceed down sequentially. Files are saved to this cloud environment and can be accessed through the folder icon on the left.

---

## <center> Textbook 3D PIB Solutions <center/>

### Wavefunction: $\ \ \ \psi_{n_x,n_y,n_z}(x,y,z)=\sqrt{\frac{8}{l_xl_yl_z}}\
\sin\bigg(\frac{n_x\pi x}{l_x}\bigg)\sin\bigg(\frac{n_y\pi y}{l_y}\
\bigg)\sin\bigg(\frac{n_z\pi z}{l_z}\bigg) $

<br>

### Energy (Ha): $\ \ \ \ \ \ E_{n_x,n_y,n_z}= \frac{\pi^2}{2}\
\bigg[\bigg(\frac{n_x}{l_x}\bigg)^2 + \bigg(\frac{n_y}{l_y}\bigg)^2 +\
 \bigg(\frac{n_z}{l_z}\bigg)^2\bigg]$

<br>

--- 
## Polycyclic Aromatic Hydrocarbon (PAH) Specifications 


PAH|Rings|Dimensions (Bohr)|$\pi$ electrons|Fuse-type
:---:|:---:|:---:|:---:|:---:
benzene   |1|   8 x 8 x 3  | 6  |linear
napthalene|2|  12 x 8 x 3 | 10 |linear
anthracene|3|  16 x 8 x 3 | 14 |linear
tetracene |4|  20 x 8 x 3 | 18 |linear
pentacene |5|  24 x 8 x 3 | 22 |linear
hexacene  |6|  28 x 8 x 3 | 26 |linear
heptacene |7|  32 x 8 x 3 | 30 |linear
---       |---|---       |--- |---
perylene  |5| 20 X 12 x 3 | 20 | non-linear
coronene  |7| 20 X 16 x 3 | 24 | non-linear

In [1]:
#@title Install/Import Packages
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=TypeError)
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown, display, clear_output
import ipywidgets as widgets
from ipywidgets import Layout
import ipyvolume as ipv
from time import sleep
from termcolor import cprint
import pythreejs
from os.path import exists
%config InlineBackend.figure_format = 'svg'


#define functions and GUI components
def psi_reg(x, y, z, q_nx=1, q_ny=1, q_nz=1,lx=1,ly=1,lz=1):
    wvfn = np.sqrt(8/(lx*ly*lz)) * \
    np.sin((q_nx*np.pi*x)/lx) * \
    np.sin((q_ny*np.pi*y)/ly) * \
    np.sin((q_nz*np.pi*z)/lz)
    return wvfn

def psi_ener(qnx, qny, qnz, lx, ly, lz):
    e_level = (4*np.pi**2/8)*((qnx/lx)**2 + (qny/ly)**2 + (qnz/lz)**2)
    return np.round(e_level,decimals=6)

def markdown_wvfn(q_nx,q_ny,q_nz,lx,ly,lz):
  subscript = f'{q_nx},{q_ny},{q_nz}'
  sqrt_denom = f'({lx})({ly})({lz})'
  display(Markdown('## $$ \\text{State: \ \ } \psi_{' + subscript + '}(x,y,z)=' +
                           '\sqrt{\\frac{8}{' + sqrt_denom + '}}' +
                           '\sin{\\Big(\\frac{' + f'{q_nx}' '\pi x}{' + f'{lx}' + '}\\Big)}' +
                           '\sin{\\Big(\\frac{' + f'{q_ny}' '\pi y}{' + f'{ly}' + '}\\Big)}' +
                           '\sin{\\Big(\\frac{' + f'{q_nz}' '\pi z}{' + f'{lz}' + '}\\Big)} \\newline $$'))

def markdown_ener(l_x, l_y, l_z):
  display(Markdown('## $$ E_{n_x,n_y,n_z} = \\frac{\pi^2}{2} \\Bigl[ ' + 
                         ' \\Bigl(\\frac{n_x}{' + f'{l_x}' + '}\\Bigl)^2 +' + 
                         ' \\Bigl(\\frac{n_y}{' + f'{l_y}' + '}\\Bigl)^2 +' + 
                         ' \\Bigl(\\frac{n_z}{' + f'{l_z}' + '}\\Bigl)^2\\Bigl] \\newline$$'))

num_elect_slider = widgets.Dropdown(options=np.arange(2,27,2),value=6,description='electrons:',disabled=False)
lx_slider = widgets.IntSlider(value=8,min=1,max=32,step=1,description='lx',disabled=False,readout_format='d',continuous_update=False)
ly_slider = widgets.IntSlider(value=8,min=1,max=32,step=1,description='ly',disabled=False,readout_format='d',continuous_update=False)
lz_slider = widgets.IntSlider(value=3,min=1,max=32,step=1,description='lz',disabled=False,readout_format='d',continuous_update=False)
lx_slider_iso = widgets.IntSlider(value=8,min=1,max=32,step=1,description='lx',disabled=False,readout_format='d',continuous_update=False)
ly_slider_iso = widgets.IntSlider(value=8,min=1,max=32,step=1,description='ly',disabled=False,readout_format='d',continuous_update=False)
lz_slider_iso = widgets.IntSlider(value=3,min=1,max=32,step=1,description='lz',disabled=False,readout_format='d',continuous_update=False)
nx_slider = widgets.IntSlider(value=1,min=1,max=10,step=1,description='nx',disabled=False,readout_format='d',continuous_update=False)
ny_slider = widgets.IntSlider(value=1,min=1,max=10,step=1,description='ny',disabled=False,readout_format='d',continuous_update=False)
nz_slider = widgets.IntSlider(value=1,min=1,max=10,step=1,description='nz',disabled=False,readout_format='d',continuous_update=False)
psi_square_check = widgets.Checkbox(value=False, description=' ',disabled=False)
psi_check_ui = widgets.HBox([widgets.Label(value='Probability density'), psi_square_check])

length_labels = widgets.Label(value='Box Lengths (Bohr): ')
qnum_labels = widgets.Label(value='Quantum Numbers: ')
qnum_ui = widgets.HBox([qnum_labels, nx_slider, ny_slider, nz_slider])
box_length_ui = widgets.HBox([length_labels, lx_slider, ly_slider, lz_slider],layout=widgets.Layout(border='solid 2px',width='%50'))
box_length_iso_ui = widgets.HBox([length_labels, lx_slider_iso, ly_slider_iso, lz_slider_iso])

filename_text_mpl = widgets.Text(description='Filename (.png): ',value='PIB_ener',style={'description_width': 'initial'})
filename_text_ipv = widgets.Text(description='Filename (.png): ',value='PIB_iso',style={'description_width': 'initial'})
save_button_mpl = widgets.Button(description='Save Image')
save_button_ipv = widgets.Button(description='Save Image')

In [2]:
#@title **Energy Diagram** (execute cell first)
def PIB_plotter(lx, ly, lz, num_elect, show=True, savefig=False, filename=None):
    ener_list = []

    for i in range(1,20):
        for j in range(1,20):
            for k in range(1,20):
                ener_list.append(((i,j,k),psi_ener(qnx=i,qny=j,qnz=k, lx=lx, ly=ly, lz=lz)))

    ener_list.sort(key=lambda x: abs(x[1]))
    ener_list = np.asarray(ener_list,dtype=object)
    ener_list[:,1] = ener_list[:,1].astype(dtype=float)

    degen_list = np.unique(ener_list[:,1],return_counts=True)

    degen_log = np.array([],dtype=int)

    for i in degen_list[1]:
        degen_log = np.append(degen_log, i*np.ones(i,dtype=int))

    degen_log = degen_log.reshape(len(degen_log),1)

    ener_list = np.hstack((ener_list,degen_log))

    #################find the unoccupied levels##############
    occ_levels = int((num_elect/ 2))
    occ_states = ener_list[0:occ_levels]
    occ_degen_accounted = np.where(occ_states[-1,1] == occ_states[:,1])[0].size
    occ_state_miss = occ_states[-1,2] - occ_degen_accounted
    if occ_state_miss > 0:
        occ_levels = occ_levels + occ_state_miss
        occ_states = ener_list[0:occ_levels]
    ##################find the occupied levesl############
    unocc_levels = 2
    unocc_states = ener_list[occ_levels:occ_levels+unocc_levels]
    unocc_degen_accounted = np.where(unocc_states[-1,1] == unocc_states[:,1])[0].size
    unocc_state_miss = unocc_states[-1,2] - unocc_degen_accounted
    if unocc_state_miss > 0:
        unocc_levels = unocc_levels + unocc_state_miss
        unocc_states = ener_list[occ_levels:occ_levels + unocc_levels]
    
    occ = []
    unocc = []
    energy_PIB = []
    for state in occ_states:
        occ.append(state[0])
        energy_PIB.append(round(state[1],8))

    for state in unocc_states:
        unocc.append(state[0])
        energy_PIB.append(round(state[1],8))
    PIBlevels = occ + unocc

    yPIB = np.array(energy_PIB)
    xPIB = np.ones(yPIB.shape[0])

    for i in range(0,len(yPIB)):
        if yPIB[i] in yPIB[:i]:
            count = list(yPIB[:i]).count(yPIB[i])
            xPIB[i] += count*0.3

    Gap_PIB = round(energy_PIB[len(occ)] - energy_PIB[len(occ)-1], 3)
    Gap_PIB_ev = round(Gap_PIB*27.2114, 3)    
    
    fig = plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(1, 1, 1)
    plt.cla()
    plt.clf()
    clear_output(wait=True)
    plt.ylabel("Energy (Ha)",labelpad=7)
    plt.scatter(xPIB[len(occ):],yPIB[len(occ):],marker=0,s=1200,linewidths=6, color='#F97306', label='virtual')
    plt.scatter(xPIB[:len(occ)],yPIB[:len(occ)],marker=0,s=1200,linewidths=6, color='green', label='occupied')
    plt.rcParams["legend.markerscale"] = 0.45
    plt.legend(loc='upper right', handlelength=3,handletextpad=.1)
    plt.xticks([])
    plt.xlim([0.3,3])
    plt.ylim([min(yPIB)-0.1,max(yPIB)+0.1])
    annotations = [str(x) for x in PIBlevels]
    for i, label in enumerate(annotations):
          if list(yPIB).count(yPIB[i])==1:
              plt.annotate(label, (xPIB[i] + 0.005, yPIB[i]),size=8)
              plt.text(xPIB[i]-0.46, yPIB[i], "{:.3f}".format(yPIB[i]),size=8)
          else:
              plt.annotate(label, (xPIB[i] - 0.25, yPIB[i] - 0.015),size=8)
              if yPIB[i] not in yPIB[:i]:
                  plt.text(xPIB[i]-0.46, yPIB[i], "{:.3f}".format(yPIB[i]),size=8)
    plt.title('3D PIB Energy Diagram')
    plt.text(0.93, 0.88, 'States: (n$_x$, n$_y$, n$_z$)',
             horizontalalignment='center', verticalalignment='center',
             transform=ax.transAxes,weight='bold')
    plt.text(0.93, 0.810,f'H-L Gap: {Gap_PIB} Ha\n              ({Gap_PIB_ev} ev)',
             size=10.5, horizontalalignment='center', verticalalignment='center',
             transform= ax.transAxes)
    ax.spines['left'].set_position(('axes', 0.16))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    plt.tight_layout()
    if savefig == True:
      plt.savefig(f'{filename}.png', dpi=800)

    if show == True:
      plt.show()
    else: 
      plt.close()

def on_click_save(b):
  PIB_plotter(lx_slider.value, ly_slider.value, lz_slider.value,
              num_elect_slider.value, show=False, savefig=True,
              filename=filename_text_mpl.value)

save_button_mpl.on_click(on_click_save)

ener_plot_out = widgets.interactive_output(PIB_plotter, {'lx':lx_slider,
                                                        'ly':ly_slider,
                                                        'lz':lz_slider,
                                                        'num_elect':num_elect_slider})

energy_display = widgets.interactive_output(markdown_ener, {'l_x':lx_slider,
                                                            'l_y':ly_slider,
                                                            'l_z':lz_slider})

display(num_elect_slider,
                box_length_ui,
                energy_display,
                Markdown('<br>'),
                ener_plot_out,
                widgets.HBox([filename_text_mpl,save_button_mpl]));

Dropdown(description='electrons:', index=2, options=(2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26), value=6)

Output()

<br>

Output()

In [3]:
#@title **Isosurface Rendering**

def isoplotter(nx_val,ny_val,nz_val,lx,ly,lz,psi_square=False,plot_save=True):

#construct 3d grid of points  
  nx_p, ny_p, nz_p = 7 * lx, 7 * ly, 7 * lz
  xp = np.linspace(0, lx, nx_p)
  yp = np.linspace(0, ly, ny_p)
  zp = np.linspace(0, lz, nz_p)
  X, Y, Z = np.meshgrid(xp, yp, zp, indexing='ij')
  psi = psi_reg(X,Y,Z,nx_val,ny_val,nz_val,lx,ly,lz)
  norm_psi = psi_reg(X,Y,Z,nx_val,ny_val,nz_val,lx,ly,lz)**2

#ipyvolume potting commands
  ipv.clear()
  fig = ipv.figure(title='PIB',width=500, height=500)
  fig.camera.type = 'OrthographicCamera'
  if psi_square:
    norm_sur = ipv.pylab.plot_isosurface(norm_psi,color='red',level=norm_psi.mean(),controls=False)
  else:
      pos_values = np.ma.array(psi, mask = psi < 0.0)
      if nx_val == ny_val == nz_val == 1:
        pos_sur = ipv.pylab.plot_isosurface(psi,color='red',level=np.sqrt(norm_psi.mean()),controls=False)
      else:
        pos_sur = ipv.pylab.plot_isosurface(psi,color='red',level=np.sqrt(norm_psi.mean()),controls=False)
        neg_sur = ipv.pylab.plot_isosurface(psi,color='blue',level=-np.sqrt(norm_psi.mean()),controls=False)

  ipv.style.box_off()
  ipv.squarelim()
  ipv.view(0,-75)
  ipv.xyzlabel('lx','ly','lz')

  ipv.show()
  
  #print(' \033[4misovalue slider (e/Bohr**3)')

def plot_saver(b):
  ipv.savefig(f'{filename_text_ipv.value}.png',width=1200,height=1200)

save_button_ipv.on_click(plot_saver)

out = widgets.interactive_output(isoplotter, {'nx_val':nx_slider,
                                              'ny_val':ny_slider,
                                              'nz_val':nz_slider,
                                              'lx':lx_slider_iso,
                                              'ly':ly_slider_iso,
                                              'lz':lz_slider_iso,
                                              'psi_square':psi_square_check})

wavefunction_display = widgets.interactive_output(markdown_wvfn, {'q_nx':nx_slider,
                                                                  'q_ny':ny_slider,
                                                                  'q_nz':nz_slider,
                                                                  'lx':lx_slider_iso,
                                                                  'ly':ly_slider_iso,
                                                                  'lz':lz_slider_iso})

#\psi_{n_x,n_y,n_z}(x,y,z)=\sqrt{\frac{8}{l_xl_yl_z}}\sin\bigg(\frac{n_x\pi x}{l_x}\bigg)\sin\bigg(\frac{n_y\pi y}{l_y}\bigg)\sin\bigg(\frac{n_z\pi z}{l_z}\bigg)

display(out,
                wavefunction_display,
                qnum_ui,
                box_length_iso_ui,
                psi_check_ui,
                widgets.HBox([filename_text_ipv,save_button_ipv]))

Output()

Output()


## **References**
 
*  Silbey, R. J.; Alberty, R. A.; Bawendi, M. G.; Papadantonakis, G. A. Physical Chemistry; John Wiley & Sons, Inc: Hoboken, **2021**. 

